<a href="https://colab.research.google.com/github/KushiBetaraju/Real-Time-Weather-Forecasting-Model-using-Python/blob/main/Real_Time_Weather_Forecasting_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Section 1: improting libraries

In [ ]:
import requests # to fetch data from api
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import pytz

In [ ]:
API_KEY = '060ef3a29f67b8a76b03213c1a121e90'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

Fetch the current weather data

In [ ]:
 '''
  def get_current_weather(city):
    url = f'{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric'
    response = requests.get(url)
    data = response.json()
    return {
        'city': data['name'],
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min': round(data['main']['temp_min']),
        'temp_max': round(data['main']['temp_max']),
        'humidity': round(data['main']['humidity']),
        'description': data['weather'][0]['description'],
        'country': data['sys']['country'],
        'wind_gust_dir':data['wind']['deg'],
        'pressure': data['main']['pressure'],
        'wind_Gust_Speed': data['wind']['speed']
    }'''

" \n def get_current_weather(city):\n   url = f'{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric'\n   response = requests.get(url)\n   data = response.json()\n   return {\n       'city': data['name'],\n       'current_temp': round(data['main']['temp']),\n       'feels_like': round(data['main']['feels_like']),\n       'temp_min': round(data['main']['temp_min']),\n       'temp_max': round(data['main']['temp_max']),\n       'humidity': round(data['main']['humidity']),\n       'description': data['weather'][0]['description'],\n       'country': data['sys']['country'],\n       'wind_gust_dir':data['wind']['deg'],\n       'pressure': data['main']['pressure'],\n       'wind_Gust_Speed': data['wind']['speed']\n   }"

In [ ]:
def get_current_weather(city):
    url = f'{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric'
    response = requests.get(url)
    data = response.json()
    return {
        'city': data['name'],
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min': round(data['main']['temp_min']),
        'temp_max': round(data['main']['temp_max']),
        'humidity': round(data['main']['humidity']),
        'description': data['weather'][0]['description'],
        'country': data['sys']['country'],
        'wind_gust_dir': data['wind']['deg'],
        'pressure': data['main']['pressure'],
        'wind_gust_speed': data['wind']['speed']  # ✅ renamed to match your prediction code
    }


2. Read Historical data

In [ ]:
def read_historical_data(filename):
  df = pd.read_csv(filename)
  df. dropna()
  df = df.drop_duplicates()
  return df

In [ ]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("zaraavagyan/weathercsv")

print("Path to dataset files:", path)
print(os.listdir(path))
weather_csv_path = os.path.join(path,'weather.csv')
weather_df = pd.read_csv(weather_csv_path)

Path to dataset files: /kaggle/input/weathercsv
['weather.csv']


3. Prepare date=a for Training

In [ ]:
'''def prepare_data(data):
  le = LabelEncoder() # Create a LabelEncoder instance
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  #Define the feature  variable and Target variables
  X = data[['MinTemp', 'MaxTemp', 'WindGutDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']] # Feature Variable
  y = data['RainTomorrow'] # Target Variable

  return X, y, le # return feature variable, target variable and the label encoder'''

"def prepare_data(data):\n  le = LabelEncoder() # Create a LabelEncoder instance\n  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])\n  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])\n\n  #Define the feature  variable and Target variables\n  X = data[['MinTemp', 'MaxTemp', 'WindGutDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']] # Feature Variable\n  y = data['RainTomorrow'] # Target Variable\n\n  return X, y, le # return feature variable, target variable and the label encoder"

In [ ]:
'''def prepare_data(data):
    # Drop rows with missing values and duplicates
    data = data.dropna().drop_duplicates()

    # Encode wind direction if it exists
    le = LabelEncoder()
    if 'WindGustDir' in data.columns:
        data['WindGustDir'] = le.fit_transform(data['WindGustDir'])

    # Define the feature and target variables
    X = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity9am', 'Pressure9am', 'Temp9am']]
    y = data['RainTomorrow']  # Target variable for classification

    return X, y, le
'''

In [ ]:
def prepare_data(data):
    # Drop rows with missing values and duplicates
    data = data.dropna().drop_duplicates()

    # Encode wind direction if it exists
    le = LabelEncoder()
    if 'WindGustDir' in data.columns:
        data['WindGustDir'] = le.fit_transform(data['WindGustDir'])

    # Define features and target
    X = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity9am', 'Pressure9am', 'Temp9am']]
    y = data['RainTomorrow']

    return X, y, le  # ✅ Must return 3 items


4. Train Rain Prediction Model using RandomForest

note:

20% data for test

 80% for training

In [ ]:
def train_rain_model(X,y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(X_train, y_train)  # Train the model

  y_pred = model.predict(X_test)  # To make predictions on test set

  print('Mean Squared  Error for Rain Model')

  print(mean_squared_error(y_test, y_pred))
  return model

In [ ]:

historical_data = read_historical_data('/content/weather.csv')
X, y, le = prepare_data(historical_data)  # ✅ This defines X, y, le
rain_model = train_rain_model(X, y)       # ✅ Now X and y are available


print("[DEBUG] X shape:", X.shape)
print("[DEBUG] y shape:", y.shape)
print("[DEBUG] Any nulls in X:", X.isnull().sum())


Mean Squared  Error for Rain Model


ValueError: could not convert string to float: 'No'

# 5. Prepare regression data

Here's the loop fills X and y:



```
# 1. First Iteration(i=0):
  >> X.append(data[feature].iloc[0]) adds 70 to X.
  >> y.append(data[feature].iloc[1]) adds 72 to y.
```


```
# 2. Second Iteration(i=1):
  >> X.append(data[Feature].iloc[1]) adds 72 to X.
  >> y.append(data[feature].iloc[2]) adds 74 to y.
```


```
# 3. Third Iteration(i=2):
  >> X.append(data[Feature].iloc[2]) adds 74 to X.
  >> y.append(data[feature].iloc[3]) adds 76 to y.
```



```
# 4. Forth Iteration(i=3):
  >> X.append(data[Feature].iloc[3]) adds 76 to X.
  >> y.append(data[feature].iloc[4]) adds 78 to y.
```


After the loop, X will be [70,72,74,76] and y will be [72,74,76,78]

In [ ]:
def prepare_regression_data(data, feature):
  X,y = [], [] #initialize list for feature and target values
  for i in range(len(data) -1):
    X.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i+1])

  X = np.array(X).reshape(-1, 1)
  y = np.array(y)
  return X,y

Train Regression Model

In [ ]:
def train_regression_model(X,y):
  model = RandomForestRegressor(n_estimators = 10, random_state = 42)
  model.fit(X, y)
  return model

Predict Future

In [ ]:
def predict_future(model, current_value):
  predictions = [current_value]

  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))

    predictions.append(next_value[0])

  return predictions[1:]

Weather Analysis Function

In [ ]:
'''def weather_view():
  city = input('\nEnter any city name: ')
  current_weather = get_current_weather(city)

  # Load historical data
  historical_data = read_historical_data('/content/weather.csv')

  # Prepare and train the rain prediction model

  X, y, le = prepare_data(historical_data)

  rain_model = train_rain_model(X,y)

  # map wind direction to campass pionts
  wind_deg = current_weather['wind_gust_dir'] % 360
  compass_points = [
      ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75,56.25)
      ("ENE", 56.25, 78.25), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
      ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
      ("SSW", 191.25, 213.25), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
      ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
      ("NNW", 326.25, 348.75)
  ]
  compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)

  compass_direction_encoded = le.transform([compass_direction ])[0] if compass_direction in le.classes_ else -1



  current_data = {
      'MinTemp': current_weather['temp_min'],
      'MaxTemp': current_weather['temp_max'],
      'WindGustDir': current_weather['temp_min'],
      'WindGustSpeed': current_weather['temp_minwind_Gust_Speed'],
      'Humidity' : current_weather['humidity'],
      'Pressure' : current_weather['pressure'],
      'Temp' : current_weather['current_temp'],
  }

  current_df = pd.DataFrame([current_data])

  # Rain Prediction

  rain_prediction = rain_model.predict(current_df)[0]

  # Prepare regression model for temperature and humidity

  X_temp, y_temp = prepare_regression_data(historical_data, "temp")

  X_hum, y_hum = prepare_regression_data(historical_data, "Humidity")

  temp_model = train_regression_model(X_hum, y_hum)

  # Predit future temperature and humidity

  future_temp = predict_future(temp_model, current_weather['temp_min'])

  future_humidity = predict_future(hum_model, current_weather['humidity'])

  # Prepare time for future predictions

  timezone = pytz.timezone('Asia/Karachi')
  now = datetime.now(timezone)
  next_hour = now + timedelta(hour = 1 )
  next_hour = now_hour.replace(minute = 0, second = 0, microsecond = 0 )

  future_times = [(next_hour + timedelta(hours = i)).strftime("%H:00") for i in range(5)]

  # Display results

  print(f"City: {city}, {current_weather['country']}")
  print(f"Current Temperature: {current_weather['current_temp']}")
  print(f"Feels like: {current_weather['feels_like']}")
  print(f"Minimum Temperature: {current_weather['temp_min']}")
  print(f"Maximum Temperature: {current_weather['temp_max']}")
  print(f"Humidity: {current_weather['humidity']}")
  print(f"Weather Prediction: {current_weather['description']}")
  print(f"Rain Preediction: {'Yes' if rain_prediction else 'No'}")


  print("\nFuture Temperature Predictions: ")

  for time, temp in zip(future_times, future_temp):
    print(f"{time}: {round(temp, 1)}°C")

  print("\nFuture Humidity Predictions: ")

  for time, humidity in zip(future_times, future_humidity):
    print(f"{time}: {round(humidity, 1)}%")

weather_view()'''

In [ ]:
# Load and clean historical data
'''
historical_data = read_historical_data('/content/weather.csv')

# ✅ Debug: Show column names
print("\n[DEBUG] Columns in your historical data:")
print(historical_data.columns.tolist())
'''

'\nhistorical_data = read_historical_data(\'/content/weather.csv\')\n\n# ✅ Debug: Show column names\nprint("\n[DEBUG] Columns in your historical data:")\nprint(historical_data.columns.tolist())\n'

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import pytz

def weather_view():
    city = input('\nEnter any city name: ')
    current_weather = get_current_weather(city)  # Function you defined to fetch API data

    # Load and clean historical data
    historical_data = read_historical_data('/content/weather.csv')

    # Prepare and train the rain prediction model
    X, y, le = prepare_data(historical_data)
    rain_model = train_rain_model(X, y)

    # Map wind direction to compass points
    wind_deg = current_weather['wind_gust_dir'] % 360
    compass_points = [
        ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75), ("N", 348.75, 360)
    ]
    compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)
    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

    # Create DataFrame for current weather (features must match training data)
    current_data = {
        'MinTemp': current_weather['temp_min'],
        'MaxTemp': current_weather['temp_max'],
        'WindGustDir': compass_direction_encoded,
        'WindGustSpeed': current_weather['wind_gust_speed'],
        'Humidity': current_weather['humidity'],
        'Pressure': current_weather['pressure'],
        'Temp': current_weather['current_temp'],
    }
    current_df = pd.DataFrame([current_data])

    # Align columns with training data
    current_df = current_df[X.columns]

    # Rain prediction
    rain_prediction = rain_model.predict(current_df)[0]

    # Prepare regression models
    X_temp, y_temp = prepare_regression_data(historical_data, "Temp")
    X_hum, y_hum = prepare_regression_data(historical_data, "Humidity")

    temp_model = train_regression_model(X_temp, y_temp)
    hum_model = train_regression_model(X_hum, y_hum)

    # Predict future temperature and humidity using current data
    current_features_for_reg = current_df[X_temp.columns]  # Ensure column match
    future_temp = predict_future(temp_model, current_features_for_reg)
    future_humidity = predict_future(hum_model, current_features_for_reg)

    # Time for future predictions
    timezone = pytz.timezone('Asia/Karachi')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)
    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

    # Display current weather
    print(f"\nCity: {city}, {current_weather['country']}")
    print(f"Current Temperature: {current_weather['current_temp']}°C")
    print(f"Feels like: {current_weather['feels_like']}°C")
    print(f"Minimum Temperature: {current_weather['temp_min']}°C")
    print(f"Maximum Temperature: {current_weather['temp_max']}°C")
    print(f"Humidity: {current_weather['humidity']}%")
    print(f"Weather Description: {current_weather['description']}")
    print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

    # Display future predictions
    print("\nFuture Temperature Predictions:")
    for time, temp in zip(future_times, future_temp):
        print(f"{time}: {round(temp, 1)}°C")

    print("\nFuture Humidity Predictions:")
    for time, humidity in zip(future_times, future_humidity):
        print(f"{time}: {round(humidity, 1)}%")

# Call the function
weather_view()



Enter any city name: England
Mean Squared  Error for Rain Model


ValueError: could not convert string to float: 'No'

In [ ]:
[DEBUG] Columns in your historical data:
['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity3pm', 'Pressure9am', 'Temperature', 'RainTomorrow']
